In [6]:
import os
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt
from models.FCN import get_FCN
from dataloaders.VOCSegmentation import load_VOCSegmentation
from tools.model_trainer import train_model

In [8]:
batch_size, crop_size = 2, (320, 480)
train_iter, test_iter = load_VOCSegmentation(batch_size, crop_size)
print(f"Size of Trian Set: {len(train_iter)}")
print(f"Size of Test Set: {len(test_iter)}")
for X, y in test_iter:
    print(X.shape, y.shape)
    break

Size of Trian Set: 557
Size of Test Set: 539
torch.Size([2, 3, 320, 480]) torch.Size([2, 320, 480])


In [9]:
net = get_FCN(num_classes=21)

In [10]:
def loss(inputs, targets):
    from torch.nn import functional as F
    return F.cross_entropy(inputs, targets, reduction='none').mean(1).mean(1)



num_epochs, lr, wd = 1, 0.001, 1e-3
device =  'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')
trainer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd)

train_model(net, train_iter, test_iter, loss, ['accuracy'], trainer, num_epochs, device, multlabel=True)


device: cpu
training on cpu
Epoch 1/1
557/557 [===============] Train Loss: 2.4672, Train Accuracy: 112725.9596, Valid Loss: 1.7059, Valid Accuracy: 120282.6308, Cost Time 268.4471 sec
Train result
loss 2.4672
final train Accuracy 112725.9596
final valid Accuracy 120282.6308
4.1498 examples/sec on cpu
268.4471 sec/epoch on cpu


In [11]:
from tools.model_tester import test_model
test_model(net, test_iter, loss, ["accuracy"], device)

Test result
Number of instances: 1078
Number of batches: 539
Avg loss of each batch: 1.708711
Accuracy: 120253.3061
Time spent: 187.52823185920715 sec
5.7485 examples/sec
0.3479 sec/batch on cpu


(1.7087105482208484, [120253.30612244898])

In [12]:
a = torch.rand((3, 4, 2))
a.numel()

24